#**Домашнее Задание "Проверка гипотез"**

**Общие моменты:**

- Обязательно гуглите всю необходимую информацию. Вы можете использовать прочие (которые не упоминались на занятиях) методы и алгоритмы для решения задач, но при условии, что Вы сможете их объяснить. Если Вы нашли какой-то интересный способ решения, то нужно в комментарий добавить ссылку на источник.

- Списывать/плагиатить строго запрещено. Можно использовать готовые решения из интернета только в случае, если Вы укажите ссылку на источник. И поясните, почему лучше так, а не каким-то стандартным (рекомендуемым) способом.

- Желательно (но не обязательно) указывать комментарии в Вашем решении. Это и Вам поможет понять, что Вы делаете. И также это поможет преверяющему в случае вопросов.

- Технологии типа ChatGPT не использовать.

- В случае каких-то вопросов или спорных ситуаций (например, после получения результатов дз), обращайтесь к ассистенту/преподавателю. И лучше пишите какие-то комментарии к своему коду, чтобы спорных ситуаций было как можно меньше.

- Каждая из задач имеет определенную «стоимость» (указана в скобках около задачи). Максимально допустимая оценка за работу — 10 баллов (+ бонус). Проверяющий имеет право снизить оценку за неэффективную реализацию или неопрятные графики.

- Сдавать ДЗ в SmartLMS!

**Успехов!**

In [ ]:
import pandas as pd
import numpy as np
import statsmodels as sm
from scipy import stats

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Параметрические критерии**

# Задание 1: z-test

Скачайте датасет Original_data_with_more_rows.csv. Он содержит данные об оценках студентов ряда университетов по 3 видам тестов. Описание данных:

- Gender: Gender of the student (male/female)
- EthnicGroup: Ethnic group of the student (group A to E)
- ParentEduc: Parent(s) education background (from some_highschool to master's degree)
- LunchType: School lunch type (standard or free/reduced)
- TestPrep: Test preparation course followed (completed or none)
- ParentMaritalStatus: Parent(s) marital status (married/single/widowed/divorced)
- PracticeSport: How often the student parctice sport (never/sometimes/regularly))
- IsFirstChild: If the child is first child in the family or not (yes/no)
- NrSiblings: Number of siblings the student has (0 to 7)
- TransportMeans: Means of transport to school (schoolbus/private)
- WklyStudyHours: Weekly self-study hours(less that 5hrs; between 5 and 10hrs; more than 10hrs)
- MathScore: math test score(0-100)
- ReadingScore: reading test score(0-100)
- WritingScore: writing test score(0-100)


In [ ]:
ord = pd.read_csv('/content/drive/MyDrive/Дата сосайенс/Original_data_with_more_rows.csv')
ord.head()

,Unnamed: 0,Gender,EthnicGroup,ParentEduc,LunchType,TestPrep,MathScore,ReadingScore,WritingScore
0,0,female,group B,bachelor's degree,standard,none,72,72,74
1,1,female,group C,some college,standard,completed,69,90,88
2,2,female,group B,master's degree,standard,none,90,95,93
3,3,male,group A,associate's degree,free/reduced,none,47,57,44
4,4,male,group C,some college,standard,none,76,78,75


## a) Посчитайте z-test для среднего значения теста по математике у девушек, родители которых закончили бакалавриат. Вес - 0.5

Пусть мы знаем, что среднее значение такого теста для всех студентов во всех университетах - 67.5. Стандартное октлонение - 13

In [ ]:
# H0: среднее по ген. совок-ти не отличается от среднего по выборке рез-тов только девушек с родителями-бакалаврами  
# H1: есть значимые отличия в средних по ген. совок-ти от среднего по выборке рез-тов только девушек с родителями-бакалаврами 
# используем формулу среднее значение по выборке - ген. среднее / (ст. откл. ген совок-ти / корень из численности выборки)

# Предварительные условия - количество выборок >= 30, должны быть известны среднее значение и стандартное отклонение совокупности
# Здесь у нас есть среднее значение и стандартное отклонение результатов теста по математике = 67,5 при стандартном отклонении 13
    
# Применим Z-тест к столбцу  ord['Gender'] == 'female') & (ord["ParentEduc"] == "bachelor's degree")]['MathScore'] данного df
#mu = μ 
mu = 67.5   # source - http://sphweb.bumc.bu.edu/otlt/MPH-Modules/BS/BS704_BiostatisticsBasics/BS704_BiostatisticsBasics3.html
std = 13
# Найдем среднее значение по выборке девушек с родителями-бакалварми в заданном фрейме данных
MeanOfSample = ord[(ord['Gender'] == 'female') & (ord["ParentEduc"] == "bachelor's degree")]['MathScore'].mean()
print("Mean Of the Sample", MeanOfSample)
SE= std/np.sqrt(len(ord[(ord['Gender'] == 'female') & (ord["ParentEduc"] == "bachelor's degree")]['MathScore']))  # стандартная ошибка через ст. откл. 
print('Standard Error:' , SE) # генеральной совокупност,и деленное на корень из размера выборки
# Z_norm_deviate =  sample_mean - population_mean /std_error
Z_norm_deviate = (MeanOfSample - mu) / SE 
print("Normal Deviate Z Value: ", Z_norm_deviate)

Mean Of the Sample 67.65222222222222
Standard Error: 0.30641293851417056
Normal Deviate Z Value:  0.4967878411413145


## b) Посчитайте p-value для полученного значения z-теста. Значимая ли разница у приведенного выборки и генеральной совокупности? Вес - 0.5

In [ ]:
# Мы будем использовать функцию scipy stats sf и проверять значимость различий на уровне значимости  5%
# Умножаем функцию sf на 2 для вычисления двустороннего p value, для двусторонней гипотезы
p_value = stats.norm.sf(abs(Z_norm_deviate))*2 
print('p value' , p_value)
if p_value > 0.05:
  print('Не удается отклонить H0')
else:
  print('Отклонить H0')

p value 0.6193386703457435
Не удается отклонить H0


С вероятностью 95% можем утверждать, что  между выборочным значением баллов за тест по матеше и средним значением генеральной совокупности существует значимая разница в Средних значениях, поэтому мы должны отклонить нулевую гипотезу H0 и принять альтернативную гипотезу: H1.

# Задание 2: одновыборочный t-test

Продолжим работу с данными о тестировании студентов. Давайте поймем, помогает ли подготовка к тесту по чтению сдавать его лучше. Мы знаем, что среднее значение для генеральной совокупности - 70. Но в этот раз мы не знаем стандартного отклонения - поэтому используем t-test


## 2a)  Посчитайте t-test для среднего значения теста по чтению у студентов, которые прошли подготовку к тесту. Вес - 0.5

Переменная TestPrep, значение completed

In [ ]:
# H0: средние значения у прошедших подготовку не отличаются от средних оценок за тест по чтению по ген совок-ти
# H1: средние значения у прошедших подготовку выше средних оценок за тест по чтению по ген совок-ти

result = stats.ttest_1samp(a=ord[ord['TestPrep'] == 'completed']['ReadingScore'], popmean=70, alternative='greater')
result

TtestResult(statistic=28.862005279791358, pvalue=1.1026838828639981e-176, df=10572)

In [ ]:
# Проверю себя, посчитав по формуле
stotkl = ord[ord['TestPrep'] == 'completed']['ReadingScore'].std()
standosh = stotkl / np.sqrt(len(ord[ord['TestPrep'] == 'completed']['ReadingScore']))
itog = (ord[ord['TestPrep'] == 'completed']['ReadingScore'].mean() - 70) / standosh
itog # действительно значение т статистики = 28.862

28.862005279791358

## 2b) Посчитайте p-value для получившегося значения и ответьте на вопрос - "Улучшает ли подгтовка к тесту его результат?". Вес - 0.5

In [ ]:
# Возьмем уровень значимости 5%. P value оказалось меньше 0.05, отклоняем гипотезу H0. Видим, что 
# различия в оценках значимы, и с вероятностью 95% мы можем утверждать, что 
# средние значения рез-тов теста по чтению у прошедших подготовку выше средних оценок за тест по чтению по ген совок-ти

if result.pvalue  > 0.05:
  print('Не удается отклонить H0')
else:
  print('Отклонить H0')

Отклонить H0


# Задание 3: парный t-test

Теперь скачаем данные Islander_data.csv. Они содержат данные от приеме пациентами лекарств от тревожности разного типа и влияение их на прохождение теста на память. Описание данных:
- first_name, last_name - имя и фамилия пациента
- age - возраст
- Happy_Sad_group настроение до приема препарата
- Dosage - доза
- Drug - препарат
- Mem_Score_Before - результат теста до приема
- Mem_Score_After - результат теста после приема
- Diff - разница тестов

In [ ]:
isl = pd.read_csv('/content/drive/MyDrive/Дата сосайенс/Islander_data.csv')
isl

,first_name,last_name,age,Happy_Sad_group,Dosage,Drug,Mem_Score_Before,Mem_Score_After,Diff
0,Bastian,Carrasco,25,H,1,A,63.5,61.2,-2.3
1,Evan,Carrasco,52,S,1,A,41.6,40.7,-0.9
2,Florencia,Carrasco,29,H,1,A,59.7,55.1,-4.6
3,Holly,Carrasco,50,S,1,A,51.7,51.2,-0.5
4,Justin,Carrasco,52,H,1,A,47.0,47.1,0.1
...,...,...,...,...,...,...,...,...,...
193,Jacob,Novak,52,H,3,T,71.3,74.3,3.0
194,Teo,Steiner,41,S,3,T,72.5,70.4,-2.1
195,Alexander,Takahashi,54,S,3,T,30.8,33.1,2.3
196,Alexandere,Takahashi,40,H,3,T,53.6,53.8,0.2


## 3a) Посчитайте, какие бывают типы препаратов и в каком объеме они представлены в данных. Вес - 0.25

In [ ]:
isl['Drug'].unique() # выясняем, каки есть типы препаратов

array(['A', 'S', 'T'], dtype=object)

In [ ]:
# смотрим,  сколько их, в абсолютных и относительных значениях. примерно равные выборки и их доли, поэтому можно сравнивать 
# влияние на одних и тех же людях до и после
num_A = isl[isl['Drug'] == 'A'].shape[0]
num_S = isl[isl['Drug'] == 'S'].shape[0]
num_T = isl[isl['Drug'] == 'T'].shape[0]
print(num_A, num_S, num_T)
share_A = round(isl[isl['Drug'] == 'A'].shape[0] / len(isl['Drug']),3)
share_S = round(isl[isl['Drug'] == 'S'].shape[0] / len(isl['Drug']),3)
share_T = round(isl[isl['Drug'] == 'T'].shape[0] / len(isl['Drug']),3)
print(share_A, share_S, share_T)

67 66 65
0.338 0.333 0.328


## 3b) Посчитатайте значение парного t-теста для изменение результата теста на память для каждого из видов препарата Вес - 0.75

In [ ]:
# берем t-тест для зависимых выборок, тк сравниваем рез-ты одних и тех же людей, просто до и после приема препарата
resA = stats.ttest_rel(isl[isl['Drug'] == 'A']['Mem_Score_Before'], isl[isl['Drug'] == 'A']['Mem_Score_After'])
resA

TtestResult(statistic=-5.8485679397308425, pvalue=1.6898179935298675e-07, df=66)

In [ ]:
resS = stats.ttest_rel(isl[isl['Drug'] == 'S']['Mem_Score_Before'], isl[isl['Drug'] == 'S']['Mem_Score_After'])
resS

TtestResult(statistic=0.1854597408842658, pvalue=0.8534459675334038, df=65)

In [ ]:
resT = stats.ttest_rel(isl[isl['Drug'] == 'T']['Mem_Score_Before'], isl[isl['Drug'] == 'T']['Mem_Score_After'])
resT

TtestResult(statistic=0.6669171283034249, pvalue=0.5072224471243232, df=64)

## 3c) Получите p-value  для каждого из полученных значений t-теста. Какие препараты влияют на память, а какие нет? Вес - 0.75


In [ ]:
# Берем все тот же уровень значимости 0.05. С вероятностью 95% мы можем утверждать, что влияние препарата А 
# на рез-ты теста на память значимо. В выборках же людей, принимавших другие два препарата, 
# различия средних оказались незначимы, согласно t-тесту для зависимых выборок
if resA.pvalue  > 0.05:
  print(resA.pvalue, 'Не удается отклонить H0')
else:
  print(resA.pvalue, 'Отклонить H0')

1.6898179935298675e-07 Отклонить H0


In [ ]:
if resS.pvalue  > 0.05:
  print(resS.pvalue, 'Не удается отклонить H0')
else:
  print(resS.pvalue, 'Отклонить H0')

0.8534459675334038 Не удается отклонить H0


In [ ]:
if resT.pvalue  > 0.05:
  print(resT.pvalue, 'Не удается отклонить H0')
else:
  print(resT.pvalue, 'Отклонить H0')

0.5072224471243232 Не удается отклонить H0


# Задание 4: двухвыборочный t-test

Теперь представим, что вы работаете аналитиком данных в компании - разработчике игр. Загрузите данные cookie_cats.csv

Ваша команда недавно на половину пользователей внедрила новое изменение в процессе игры: теперь пользователей в самом начале попадает не к воротам на 30 уровне, а к воротам на 40 уровне (что бы это не значило). 

Ваша задача - понять, какой эффект на бизнес это дало. Описание данных:
- userid - id пользователя
- version - группа пользователей: gate_30 или gate_40
- sum_gamerounds - количество сыгранных раундов за первые 14 дней
- retention_1 - флаг того, что пользователь вернется в игру через 1 день после скачивания
- retention_7 - флаг того, что пользователь вернется в игру через 7 дней после скачивания

In [ ]:
cc = pd.read_csv('/content/drive/MyDrive/Дата сосайенс/cookie_cats.csv')
cc

,userid,version,sum_gamerounds,retention_1,retention_7
0,116,gate_30,3,False,False
1,337,gate_30,38,True,False
2,377,gate_40,165,True,False
3,483,gate_40,1,False,False
4,488,gate_40,179,True,True
...,...,...,...,...,...
90184,9999441,gate_40,97,True,False
90185,9999479,gate_40,30,False,False
90186,9999710,gate_30,28,True,False
90187,9999768,gate_40,51,True,False


## 4a) Замените в колонках retention_1 и retention_7 значения True на 1, а знаечния False на 0. Вес - 0.25

Это позволит удобнее считать вероятность возвращения

In [ ]:
cc['retention_1'] = cc['retention_1'].apply(lambda x: 1 if x == True else 0)
cc['retention_7'] = cc['retention_7'].apply(lambda x: 1 if x == True else 0)
cc

,userid,version,sum_gamerounds,retention_1,retention_7
0,116,gate_30,3,0,0
1,337,gate_30,38,1,0
2,377,gate_40,165,1,0
3,483,gate_40,1,0,0
4,488,gate_40,179,1,1
...,...,...,...,...,...
90184,9999441,gate_40,97,1,0
90185,9999479,gate_40,30,0,0
90186,9999710,gate_30,28,1,0
90187,9999768,gate_40,51,1,0


## 4b) Посчитайте среднее по двум группам для каждой из переменных sum_gamerounds, retention_1, retention_7. Вес - 0.5

In [ ]:
mn1_30 = round(cc[cc['version'] == 'gate_30']['retention_1'].mean(),3)
mn1_40 = round(cc[cc['version'] == 'gate_40']['retention_1'].mean(),3)

mn7_30 = round(cc[cc['version'] == 'gate_30']['retention_7'].mean(),3)
mn7_40 = round(cc[cc['version'] == 'gate_40']['retention_7'].mean(),3)

mn_sg_30 = round(cc[cc['version'] == 'gate_30']['sum_gamerounds'].mean(),3)
mn_sg_40 = round(cc[cc['version'] == 'gate_40']['sum_gamerounds'].mean(),3)

print(' средние по retention_1 ', mn1_30,'   ', mn1_40,'\n',
      'средние по retention_7 ', mn7_30,'    ', mn7_40,'\n',
      'средние по gamerounds ', mn_sg_30, '   ', mn_sg_40)

 средние по retention_1  0.448     0.442 
 средние по retention_7  0.19      0.182 
 средние по gamerounds  52.456     51.299


## 4с) Посчитайте двухвыборочный t-test для каждой из переменных выше. Вес - 0.5

In [ ]:
# для переменной retention_1 в играх с gate_30 и с gate_40
# H0: различия в среднем ретеншне 1 дня при gate_30 и gate_40 незначимы
# H1: различия в среднем ретеншне 1 дня при gate_30 и gate_40 значимы
gate30 = cc[cc['version'] == 'gate_30']['retention_1']
gate40 = cc[cc['version'] == 'gate_40']['retention_1']

result1_2samp = stats.ttest_ind(a=gate30, b=gate40, equal_var=False)
result1_2samp

Ttest_indResult(statistic=1.7840774867039824, pvalue=0.07441443713953834)

In [ ]:
# для переменной retention_7 в играх с gate_30 и с gate_40
# H0: различия в среднем ретеншне 7 дня при gate_30 и gate_40 незначимы
# H1: различия в среднем ретеншне 7 дня при gate_30 и gate_40 значимы
gate30_ret7 = cc[cc['version'] == 'gate_30']['retention_7']
gate40_ret7 = cc[cc['version'] == 'gate_40']['retention_7']

result2_2samp = stats.ttest_ind(a=gate30_ret7, b=gate40_ret7, equal_var=False)
result2_2samp

Ttest_indResult(statistic=3.164028946774232, pvalue=0.001556530181006652)

In [ ]:
# для переменной sum_gamerounds в играх с gate_30 и с gate_40
# H0: различия в количестве проведенным польз-лями раундов игры при gate_30 и gate_40 незначимы
# H1: различия в количестве польз-лями раундов игры при gate_30 и gate_40 значимы
gate30_sg = cc[cc['version'] == 'gate_30']['sum_gamerounds']
gate40_sg = cc[cc['version'] == 'gate_40']['sum_gamerounds']

result3_2samp = stats.ttest_ind(a=gate30_sg, b=gate40_sg, equal_var=False)
result3_2samp

Ttest_indResult(statistic=0.8854374331270672, pvalue=0.37592438409326223)

## 4d) Посчитайте p-value для полученных значений t-теста. Как новое изменение влияет на пользователей? Вес - 0.5

In [ ]:
# Проверим, значимы ли различия в средних, выбрав уровень значимости 5%. 

if result1_2samp.pvalue > 0.05:
  print('Не удается отклонить H0')
else:
  print('Отклонить H0')

# Изменения в значении ретеншна 1го дня с введением изменений в процессе игры незначимы с вер-ю 95%. 
# Т.е. здесь мы не можем с вероятностью 95% утверждать, что изменения в ретеншне первого дня вызваны 
# изменениями в процессе игры со стороны разработчиков. 

Не удается отклонить H0


In [ ]:
if result2_2samp.pvalue > 0.05:
  print('Не удается отклонить H0')
else:
  print('Отклонить H0')
# Мы можем утверждать с вероятностью 95%, что изменения в ретеншне седьмого дня (причем в меньшую сторону;( ) вызваны
# изменениями в процессе игры со стороны разработчиков. 

Отклонить H0


In [ ]:
if result3_2samp.pvalue > 0.05:
  print('Не удается отклонить H0')
else:
  print('Отклонить H0')
# Изменения в количестве игровых раундов с введением изменений в процессе игры незначимы с вер-ю 95%. 

Не удается отклонить H0


# **Непараметрические критерии**

# Задание 5: Критерий согласия хи-квадрат

Исследуем предпочтения еды у разных национальностей. Скачайте данные Food_Preference.csv. В них приведены результаты опроса некоторой выборки людей о их вкусовых предпочтениях

Описание данных:
- Timestamp: время опроса
- Participant_ID: id участиника
- Gender: пол
- Nationality: национальность
- Age: возраст
- Food: предпочтение по типу еды
- Juice: предпочтение по напитакам
- Dessert: предпочтение по дессертам

In [ ]:
fp = pd.read_csv('/content/drive/MyDrive/Дата сосайенс/Food_Preference.csv')
fp

,Timestamp,Participant_ID,Gender,Nationality,Age,Food,Juice,Dessert
0,2019/05/07 2:59:13 PM GMT+8,FPS001,Male,Indian,24,Traditional food,Fresh Juice,Maybe
1,2019/05/07 2:59:45 PM GMT+8,FPS002,Female,Indian,22,Western Food,Carbonated drinks,Yes
2,2019/05/07 3:00:05 PM GMT+8,FPS003,Male,Indian,31,Western Food,Fresh Juice,Maybe
3,2019/05/07 3:00:11 PM GMT+8,FPS004,Female,Indian,25,Traditional food,Fresh Juice,Maybe
4,2019/05/07 3:02:50 PM GMT+8,FPS005,Male,Indian,27,Traditional food,Fresh Juice,Maybe
...,...,...,...,...,...,...,...,...
283,2019/05/10 9:24:00 AM GMT+8,FPS284,Male,Indian,27,Western Food,Fresh Juice,Yes
284,2019/05/10 9:32:54 AM GMT+8,FPS285,Male,Indian,24,Traditional food,Fresh Juice,Yes
285,2019/05/10 12:09:17 PM GMT+8,FPS286,Male,Indian,25,Traditional food,Fresh Juice,Yes
286,2019/05/10 12:52:17 PM GMT+8,FPS287,Male,Indian,27,Traditional food,Fresh Juice,Yes


## 5a) Посчитайте, какие есть национальности и в каком количестве. А также какие есть возможные варианты предпочтений в еде. Вес - 0.5 балла

In [ ]:
sorted(fp['Nationality'].unique())

['Algerian ',
 'Canadian',
 'China',
 'Indian',
 'Indonesain',
 'Indonesia',
 'Indonesian',
 'Indonesian ',
 'Japan',
 'Korean ',
 'MALAYSIAN',
 'MY',
 'Malaysia',
 'Malaysia ',
 'Malaysian',
 'Malaysian ',
 'Maldivian',
 'Maldivian ',
 'Mauritian',
 'Muslim',
 'Nigerian ',
 'Pakistan',
 'Pakistani',
 'Pakistani ',
 'Seychellois',
 'Tanzanian',
 'Yemen']

In [ ]:
fp['Nationality'] = fp['Nationality'].apply(lambda x: x.strip())
fp['Nationality'] = fp['Nationality'].apply(lambda x: 'Malaysian' if x == 'MY' or
                                            x == 'MALAYSIAN' or x == 'Malaysia' else x)
fp['Nationality'] = fp['Nationality'].apply(lambda x: 'Indonesian' if x == 'Indonesain' or
                                            x == 'Indonesia' else x)
fp['Nationality'] = fp['Nationality'].apply(lambda x: 'Pakistani' if x ==  'Pakistan' else x)

In [ ]:
# Уникальные национальности есть:
sorted(fp['Nationality'].unique())

['Algerian',
 'Canadian',
 'China',
 'Indian',
 'Indonesian',
 'Japan',
 'Korean',
 'Malaysian',
 'Maldivian',
 'Mauritian',
 'Muslim',
 'Nigerian',
 'Pakistani',
 'Seychellois',
 'Tanzanian',
 'Yemen']

In [ ]:
fp.groupby('Nationality', as_index= False)['Nationality'].value_counts() #Количество людей разных нац-тей в выборке

,Nationality,count
0,Algerian,1
1,Canadian,1
2,China,1
3,Indian,241
4,Indonesian,10
5,Japan,2
6,Korean,1
7,Malaysian,16
8,Maldivian,3
9,Mauritian,1


In [ ]:
pref = sorted(fp['Food'].unique())
howmany = len(sorted(fp['Food'].unique()))
print(pref,'\n', howmany)

['Traditional food', 'Western Food'] 
 2


## 5b) Выберите самую частую национальность. Используйте критерий согласия хи-квадрат чтобы ответить на вопрос: "Одинаково ли эта национальность любит все типы еды?". Вес - 1 балл

In [ ]:
# убираю из выборки рандомно одного индуса с помощью drop, чтобы их было четное количество, 
# и expected число любителей вестерн кухни не было сто двадцать плюс пол-индуса:)
df = fp.drop(index = 286)
df

,Timestamp,Participant_ID,Gender,Nationality,Age,Food,Juice,Dessert
0,2019/05/07 2:59:13 PM GMT+8,FPS001,Male,Indian,24,Traditional food,Fresh Juice,Maybe
1,2019/05/07 2:59:45 PM GMT+8,FPS002,Female,Indian,22,Western Food,Carbonated drinks,Yes
2,2019/05/07 3:00:05 PM GMT+8,FPS003,Male,Indian,31,Western Food,Fresh Juice,Maybe
3,2019/05/07 3:00:11 PM GMT+8,FPS004,Female,Indian,25,Traditional food,Fresh Juice,Maybe
4,2019/05/07 3:02:50 PM GMT+8,FPS005,Male,Indian,27,Traditional food,Fresh Juice,Maybe
...,...,...,...,...,...,...,...,...
282,2019/05/10 9:13:05 AM GMT+8,FPS283,Male,Indian,27,Traditional food,Fresh Juice,No
283,2019/05/10 9:24:00 AM GMT+8,FPS284,Male,Indian,27,Western Food,Fresh Juice,Yes
284,2019/05/10 9:32:54 AM GMT+8,FPS285,Male,Indian,24,Traditional food,Fresh Juice,Yes
285,2019/05/10 12:09:17 PM GMT+8,FPS286,Male,Indian,25,Traditional food,Fresh Juice,Yes


In [ ]:
# Выбираем уровень значимости и кол-во степеней свободы для 2 категорий
alpha = 0.05
ddof = 2 - 1 
chi2theor = 3.84
# сранивать будем с хи-квадрат = 3.84

In [ ]:
indians_love_western = len(df[df['Nationality'] == 'Indian'])/2 
indians_love_traditional = len(df[df['Nationality'] == 'Indian'])/2
# ожидаем: одинаковое кол-во индусов из выборки любит и западную, и традиционную еду
exp = [indians_love_western, indians_love_traditional]
exp

[120.0, 120.0]

In [ ]:
indians_really_love_western = df[(df['Nationality'] == 'Indian') & (df['Food'] == 'Western Food')].shape[0]
indians_really_love_traditional = df[(df['Nationality'] == 'Indian') & (df['Food'] == 'Traditional food')].shape[0]
obs = [indians_really_love_western, indians_really_love_traditional]
obs

[37, 203]

In [ ]:
chi2 = ((obs[0] - exp[0])**2 / exp[0]) +  ((obs[1] - exp[1])**2 / exp[1])
chi2

114.81666666666666

In [ ]:
chi2 > chi2theor #значение хи-квадрат попадает в критическую область, значит мы можем отвергнуть H0
# о том, что наблюдаемые частоты не отличаются от распределения частот теоретических. Индусы не любят все типы еды
# одинаково, а больше предпочитают традиционную

True

In [ ]:
# проверо4ка значимости через встроенную функцию
ex_ob = pd.DataFrame()
ex_ob['Кухня'] = ['Western Food', 'Traditional Food']
ex_ob['exp'] = [exp[0], exp[1]]
ex_ob['obs'] = [obs[0], obs[1]]
ex_ob

,Кухня,exp,obs
0,Western Food,120.0,37
1,Traditional Food,120.0,203


In [ ]:
chisqres = stats.chisquare(ex_ob["obs"], ex_ob["exp"]) 

In [ ]:
chisqres.pvalue > 0.05 # значение p, равное или меньшее 0.05, указывает на то, что нулевую гипотезу можно отвергнуть

False

# Задание 6: Критерий независимости хи-квадрат

## 6a) Используйте критерий независимости хи-квадрат, чтобы для той же национальности ответить на вопрос: "Одинаковые ли вкусовые предпочтения по еде у мужчин и женщин?". Вес - 1.5 балла

In [ ]:
df[df['Nationality'] == 'Indian']['Gender'].unique()

array(['Male', 'Female', nan], dtype=object)

In [ ]:
df.dropna(inplace= True)
df

,Timestamp,Participant_ID,Gender,Nationality,Age,Food,Juice,Dessert
0,2019/05/07 2:59:13 PM GMT+8,FPS001,Male,Indian,24,Traditional food,Fresh Juice,Maybe
1,2019/05/07 2:59:45 PM GMT+8,FPS002,Female,Indian,22,Western Food,Carbonated drinks,Yes
2,2019/05/07 3:00:05 PM GMT+8,FPS003,Male,Indian,31,Western Food,Fresh Juice,Maybe
3,2019/05/07 3:00:11 PM GMT+8,FPS004,Female,Indian,25,Traditional food,Fresh Juice,Maybe
4,2019/05/07 3:02:50 PM GMT+8,FPS005,Male,Indian,27,Traditional food,Fresh Juice,Maybe
...,...,...,...,...,...,...,...,...
282,2019/05/10 9:13:05 AM GMT+8,FPS283,Male,Indian,27,Traditional food,Fresh Juice,No
283,2019/05/10 9:24:00 AM GMT+8,FPS284,Male,Indian,27,Western Food,Fresh Juice,Yes
284,2019/05/10 9:32:54 AM GMT+8,FPS285,Male,Indian,24,Traditional food,Fresh Juice,Yes
285,2019/05/10 12:09:17 PM GMT+8,FPS286,Male,Indian,25,Traditional food,Fresh Juice,Yes


In [ ]:
df[df['Nationality'] == 'Indian'].shape[0] #после удаление пропусков по Gender индусов снова стало нечетное число
# дропнем рандомно снова какую-нибудь строку из выборки индусов

237

In [ ]:
df = df.drop(index = 287)

In [ ]:
# H0: В генеральной совокупности пропорции распределения предпочитаемого 
# типа еды для женщин не отличаются от пропорций распределения предпочитаемого типа еды для мужчин
# H1: В генеральной совокупности пропорции распределения предпочитаемого 
# типа еды для женщин отличаются от пропорций распределения предпочитаемого типа еды для мужчин

fem_love_western = df[(df['Nationality'] == 'Indian') & (df['Gender'] == 'Female') & (df['Food'] == 'Western Food')].shape[0]
fem_love_tr = df[(df['Nationality'] == 'Indian') & (df['Gender'] == 'Female') & (df['Food'] == 'Traditional food')].shape[0]
male_love_western = len(df[(df['Nationality'] == 'Indian') & (df['Gender'] == 'Male') & (df['Food'] == 'Western Food')])
male_love_tr = len(df[(df['Nationality'] == 'Indian') & (df['Gender'] == 'Male') & (df['Food'] == 'Traditional food')])

alpha = 0.05
ddof = (2-1)*(2-1)
chi2theor = 3.84
fe_ma = pd.DataFrame()
fe_ma['Кухня'] = ['Western Food', 'Traditional Food', 'Сумма по полу']
fe_ma['Female'] = [fem_love_western, fem_love_tr,
                   fem_love_western+fem_love_tr]
fe_ma['Male'] = [male_love_western, 
                 male_love_tr,
                 male_love_western+male_love_tr]
fe_ma['Summa'] = [fem_love_western+male_love_western, male_love_tr + fem_love_tr, 
                fem_love_western+male_love_western + male_love_tr + fem_love_tr]
fe_ma

,Кухня,Female,Male,Summa
0,Western Food,17,20,37
1,Traditional Food,132,67,199
2,Сумма по полу,149,87,236


In [ ]:
pref_western = fe_ma['Summa'][0]/fe_ma['Summa'][2]
pref_trad = fe_ma['Summa'][1]/fe_ma['Summa'][2]
print(pref_western, pref_trad)

0.15677966101694915 0.8432203389830508


In [ ]:
# составляем таблицу с теоретическими (ожидаемыми) частотами
theor_fem_love_western = fe_ma['Female'][2]*pref_western
theor_male_love_western = fe_ma['Male'][2]*pref_western
theor_fem_love_trad = fe_ma['Female'][2]*pref_trad
theor_male_love_trad = fe_ma['Male'][2]*pref_trad

th_fe_ma = pd.DataFrame()
th_fe_ma['Кухня'] = ['Western Food', 'Traditional Food', 'Сумма по полу']
th_fe_ma['Female'] = [theor_fem_love_western, theor_fem_love_trad,
                   theor_fem_love_western+theor_fem_love_trad]
th_fe_ma['Male'] = [theor_male_love_western, 
                 theor_male_love_trad,
                 theor_male_love_western+theor_male_love_trad]
th_fe_ma['Summa'] = [theor_fem_love_western+theor_male_love_western, theor_male_love_trad + theor_fem_love_trad, 
                theor_fem_love_western+theor_male_love_western + theor_male_love_trad + theor_fem_love_trad]
th_fe_ma

,Кухня,Female,Male,Summa
0,Western Food,23.360169,13.639831,37.0
1,Traditional Food,125.639831,73.360169,199.0
2,Сумма по полу,149.000000,87.000000,236.0


In [ ]:
chi2fe_male = ((fe_ma['Female'][0] - th_fe_ma['Female'][0])**2  / th_fe_ma['Female'][0]) + (
    (fe_ma['Female'][1] - th_fe_ma['Female'][1])**2  / th_fe_ma['Female'][1]) + (
        (fe_ma['Male'][0] - th_fe_ma['Male'][0])**2  / th_fe_ma['Male'][0]) + (
            (fe_ma['Male'][1] - th_fe_ma['Male'][1])**2  / th_fe_ma['Male'][1])
chi2fe_male

5.570742265235328

In [ ]:
chi2fe_male > chi2theor #нулевая гипотеза не отвергается. В генеральной совокупности пропорции распределения предпочитаемого 
# типа еды для женщин не отличаются от пропорций распределения предпочитаемого типа еды для мужчин

True

## 6b) Теперь поймите, есть ли различия в предпочтениях в дессертах у мужчин и женщин той же национальности. Также используйте хи-квадрат. Вес - 1.5 балла

In [ ]:
# H0: В генеральной совокупности пропорции распределения предпочитаемого 
# типа еды для женщин не отличаются от пропорций распределения предпочитаемого типа еды для мужчин
# H1: В генеральной совокупности пропорции распределения предпочитаемого 
# типа еды для женщин отличаются от пропорций распределения предпочитаемого типа еды для мужчин

fem_no = df[(df['Nationality'] == 'Indian') & (df['Gender'] == 'Female') & (df['Dessert'] == 'No')].shape[0]
fem_yes = df[(df['Nationality'] == 'Indian') & (df['Gender'] == 'Female') & (df['Dessert'] == 'Yes')].shape[0]
fem_maybe = df[(df['Nationality'] == 'Indian') & (df['Gender'] == 'Female') & (df['Dessert'] == 'Maybe')].shape[0]
male_no = df[(df['Nationality'] == 'Indian') & (df['Gender'] == 'Male') & (df['Dessert'] == 'No')].shape[0]
male_yes = df[(df['Nationality'] == 'Indian') & (df['Gender'] == 'Male') & (df['Dessert'] == 'Yes')].shape[0]
male_maybe = df[(df['Nationality'] == 'Indian') & (df['Gender'] == 'Male') & (df['Dessert'] == 'Maybe')].shape[0]

alpha = 0.05
ddof = (3-1)*(2-1)

desserts_obs = pd.DataFrame()
desserts_obs['Preference dess'] = ['Yes', 'No', 'Maybe', 'Сумма']
desserts_obs['Female'] = [fem_yes, fem_no, fem_maybe, fem_yes+fem_no+fem_maybe]
desserts_obs['Male']= [male_yes, male_no, male_maybe, male_yes+male_no+male_maybe]
desserts_obs['Sum'] = [fem_yes+male_yes, fem_no+male_no, fem_maybe+male_maybe, fem_yes+fem_no+fem_maybe+male_yes+male_no+male_maybe]
desserts_obs

,Preference dess,Female,Male,Sum
0,Yes,49,38,87
1,No,34,12,46
2,Maybe,66,37,103
3,Сумма,149,87,236


In [ ]:
for_yes = desserts_obs['Sum'][0]/desserts_obs['Sum'][3]
for_no = desserts_obs['Sum'][1]/desserts_obs['Sum'][3]
for_maybe = desserts_obs['Sum'][2]/desserts_obs['Sum'][3]
print(for_yes, for_no, for_maybe)

0.3686440677966102 0.19491525423728814 0.4364406779661017


In [ ]:
# составляем таблицу с теоретическими (ожидаемыми) частотами
theor_fem_yes = desserts_obs['Female'][3]*for_yes
theor_fem_no = desserts_obs['Female'][3]*for_no
theor_fem_maybe = desserts_obs['Female'][3]*for_maybe
theor_male_yes = desserts_obs['Male'][3]*for_yes
theor_male_no = desserts_obs['Male'][3]*for_no
theor_male_maybe = desserts_obs['Male'][3]*for_maybe

theor_dess_obs = pd.DataFrame()
theor_dess_obs['Preference dess'] = ['Yes', 'No', 'Maybe', 'Summa']
theor_dess_obs['Female'] = [theor_fem_yes, theor_fem_no, theor_fem_maybe, theor_fem_yes+theor_fem_no+theor_fem_maybe]
theor_dess_obs['Male']= [theor_male_yes, theor_male_no, theor_male_maybe, theor_male_yes+theor_male_no+theor_male_maybe]
theor_dess_obs['Sum'] = [theor_fem_yes+theor_male_yes, theor_fem_no+theor_male_no, theor_fem_maybe+theor_male_maybe, 
                       theor_fem_yes+theor_fem_no+theor_fem_maybe+theor_male_yes+theor_male_no+theor_male_maybe]
theor_dess_obs

,Preference dess,Female,Male,Sum
0,Yes,54.927966,32.072034,87.0
1,No,29.042373,16.957627,46.0
2,Maybe,65.029661,37.970339,103.0
3,Summa,149.000000,87.000000,236.0


In [ ]:
chi2fe_dess = ((desserts_obs['Female'][0] - theor_dess_obs['Female'][0])**2  / theor_dess_obs['Female'][0]) + (
    (desserts_obs['Female'][1] - theor_dess_obs['Female'][1])**2  / theor_dess_obs['Female'][1])+ (
        (desserts_obs['Female'][2] - theor_dess_obs['Female'][2])**2  / theor_dess_obs['Female'][2]) + (
            (desserts_obs['Male'][0] - theor_dess_obs['Male'][0])**2  / theor_dess_obs['Male'][0]) + (
                (desserts_obs['Male'][1] - theor_dess_obs['Male'][1])**2  / theor_dess_obs['Male'][1] + (
                    (desserts_obs['Male'][2] - theor_dess_obs['Male'][2])**2  / theor_dess_obs['Male'][2]))
chi2fe_dess

4.070384540067486

In [ ]:
chi2_critical = 5.99
chi2fe_dess > chi2_critical # Нулевая гипотеза отвергается. В генеральной совокупности 
# зависимость между биологическим полом и предпочтением в десертах существует.

False